In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [14]:
from functools import partial

In [320]:
def random_partition(n, k, rand_generator):
    ''' Generate k values that sum to n '''
    p = []
    for i in range(k):
        v = rand_generator()
        p.append(min(n, v) if i < k-1 else n)
        n -= p[-1]
    return p
        
def generate_dataset(nodes, levels, total_supplies, total_demands, 
                     transp_costs, random_state=None):
    # Generate number of nodes per level
#     n_nodes = [nodes // levels] * (levels - 1) + \
#               [nodes % levels + nodes // levels]
    offset = int(0.25 * (nodes // levels) + 1)
    n_nodes = random_partition(nodes, levels, 
                    partial(np.random.randint, low=max(1,nodes // levels - offset),
                                              high=nodes // levels + offset))
    assert sum(n_nodes) == nodes # partition
    if random_state:
        np.random.seed(random_state)
    
    # Generate supplies for each item
    supplies = []
    for total_supply in total_supplies: 
        supplies.append(random_partition(
                total_supply, n_nodes[0], 
                partial(np.random.poisson, lam=total_supply // n_nodes[0])))
        assert sum(supplies[-1]) == total_supply # partition
    
    # Generate demands for each item
    demands = []
    for total_demand in total_demands:
        demands.append(random_partition(
                total_demand, n_nodes[-1],
                partial(np.random.poisson, lam=total_demand // n_nodes[-1])))
        assert sum(demands[-1]) == total_demand # partition
        
    # Generate costs
    costs = [np.random.randint(*transp_costs, (n_nodes[i], n_nodes[i+1])) \
                for i in range(levels-1)]
    
    # Generate capacities
    flow = max(sum(total_supplies), sum(total_demands))
    capacities = [list(np.sum(supplies, axis=0))] + \
                 [random_partition(flow + np.random.poisson(int(0.1 * flow)), n, 
                     partial(np.random.poisson, lam=flow // n)) \
                     for n in n_nodes[1:-1]] + \
                 [list(np.sum(demands, axis=0))]
    
    dummy_supplies = [max(0, d - s) for s,d in zip(total_supplies, total_demands)]
    dummy_demands = [max(0, s - d) for s,d in zip(total_supplies, total_demands)]
    
    # Supply-demand + dummies (supply-demand)
    supplies = [sup + [dum] for sup,dum in zip(supplies, dummy_supplies)]
    demands = [dem + [dum] for dem,dum in zip(demands, dummy_demands)]

    return n_nodes, supplies, demands, costs, capacities

In [372]:
n_nodes, supplies, demands, costs, capacities = generate_dataset(
                nodes=10, levels=4, total_supplies=[25,30], 
                total_demands=[28,25], transp_costs=(10,40))

In [373]:
n_nodes

[2, 1, 2, 5]

In [374]:
supplies

[[14, 11, 3], [16, 14, 0]]

In [375]:
demands

[[5, 4, 4, 3, 12, 0], [4, 6, 9, 5, 1, 5]]

In [376]:
capacities

[[30, 25], [67], [17, 43], [9, 10, 13, 8, 13]]

In [377]:
[cost.shape for cost in costs]

[(2, 1), (1, 2), (2, 5)]

In [378]:
n_nodes

[2, 1, 2, 5]

In [379]:
' '.join([f'T{i+1}{j+1}' for i in range(len(n_nodes[1:-1])) \
                        for j in range(n_nodes[1:-1][i])])

'T11 T21 T22'

In [380]:
supply_trans = ' '.join([f'(S{j+1},T1{k+1})' \
                         for j in range(n_nodes[0]) \
                         for k in range(n_nodes[1])])
trans_trans = ' '.join([f'(T{i+1}{j+1},T{i+2}{k+1})' \
                         for i in range(len(n_nodes[1:-1])-1) \
                         for j in range(n_nodes[1:-1][i]) \
                         for k in range(n_nodes[1:-1][i+1])])
trans_demand = ' '.join([f'(T{len(n_nodes[1:-1])}{j+1},D{k+1})' \
                         for j in range(n_nodes[-2]) \
                         for k in range(n_nodes[-1])])

In [381]:
supply_trans

'(S1,T11) (S2,T11)'

In [382]:
trans_demand

'(T21,D1) (T21,D2) (T21,D3) (T21,D4) (T21,D5) (T22,D1) (T22,D2) (T22,D3) (T22,D4) (T22,D5)'

In [383]:
trans_trans

'(T11,T21) (T11,T22)'

In [386]:
def generate_ampl(n_nodes, supplies, demands, costs, capacities):
    with open('data/model.mod', 'r') as f_model, \
         open('data/model_data.mod', 'w') as fout:
        print(f_model.read(), file=fout)
        print('data;', file=fout)
        
        # Nodes
        items = ' '.join([f'I{i+1}' for i in range(len(supplies))])
        print(f'set I := {items};', file=fout)
        supply = ' '.join([f'S{i+1}' for i in range(n_nodes[0])])
        trans = ' '.join([f'T{i+1}{j+1}' for i in range(len(n_nodes[1:-1])) \
                                         for j in range(n_nodes[1:-1][i])])
        demand = ' '.join([f'D{i+1}' for i in range(n_nodes[-1])])
        print(f'set ST := {supply} {trans};', file=fout)
        print(f'set D := {demand};', file=fout)
        print(f'set DU := dummy_sup dummy_dem;\n', file=fout)
        
        # Edges
        print(f'set E := ', file=fout)
        supply_trans = ' '.join([f'(S{j+1},T1{k+1})' \
                                 for j in range(n_nodes[0]) \
                                 for k in range(n_nodes[1])])
        trans_trans = ' '.join([f'(T{i+1}{j+1},T{i+2}{k+1})' \
                                 for i in range(len(n_nodes[1:-1])-1) \
                                 for j in range(n_nodes[1:-1][i]) \
                                 for k in range(n_nodes[1:-1][i+1])])
        trans_demand = ' '.join([f'(T{len(n_nodes[1:-1])}{j+1},D{k+1})' \
                                 for j in range(n_nodes[-2]) \
                                 for k in range(n_nodes[-1])])
        print(f'   {supply_trans}', file=fout)
        print(f'   {trans_trans}', file=fout)
        print(f'   {trans_demand};\n', file=fout)
        print(f'set EDU := ', file=fout)
        supply_dummy = ' '.join(f'(S{j+1},dummy_dem)' \
                                 for j in range(n_nodes[0]))
        dummy_demand = ' '.join(f'(dummy_sup,D{j+1})' \
                                 for j in range(n_nodes[-1]))
        print(f'   {supply_trans}', file=fout)
        print(f'   {supply_dummy}', file=fout)
        print(f'   {trans_trans}', file=fout)
        print(f'   {trans_demand}', file=fout)
        print(f'   {dummy_demand};\n', file=fout)
        
        # Params
        print(f'param transp_cost := ', file=fout)
        

In [387]:
generate_ampl(n_nodes, supplies, demands, costs, capacities)